# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

The dataset we are using here is the "hottest" dataset in Kaggle for credit card fraud detection.<br>
The target of problem is to identify based on the given features if a transaction is fraudulent or not. <br>
The dataset contains only numerical input variables which are a result of a PCA transformation. <br>
This means that the datset does not require any cleaning, nomralisation or any kind of encoding. <br>

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-fraud-credit-exp'

experiment=Experiment(ws, experiment_name)

In the cell below we are creating a cluster to run our AutoML experiment

In [3]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In the cell below we read the data if they are already saved in the Workspace <br>
If they are not then we can download the data from a URL <br>
Here the URL thought doesn't work properly so it's in comments <br>

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "fraud-capstone"
description_text = "Credit Card Fraud Detection Data"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

# if not found:
#         # Create AML Dataset and register it into Workspace
#         example_data = 'https://drive.google.com/file/d/1qEMDzGfw-iT4H127-ph3h_gmUfrSocpQ/view?usp=sharing'
#         dataset = Dataset.Tabular.from_delimited_files(example_data)        
#         #Register Dataset in Workspace
#         dataset = dataset.register(workspace=ws,
#                                    name=key,
#                                    description=description_text)


df = dataset.to_pandas_dataframe()
display(df.describe())
display(df.head())

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.841366,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.958151e-16,-3.147640e-15,...,1.471982e-16,8.042109e-16,5.282450e-16,4.458267e-15,1.426896e-15,1.701640e-15,-3.671606e-16,-1.218152e-16,88.349619,0.001727
std,47488.228330,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.250000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84691.500000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.750000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I am giving the experiment a timeout after 30 minutes <br>
It can run up to 5 Runs at the same time <br>
Because the dataset is highly imbalanced we use AUC and its weighted <br>
The problem is a binary classification <br>
XGboostClassifier was giving me an error so I removed it <br>

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Class",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             blocked_models=["XGBoostClassifier"],
                             **automl_settings
                            )

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-fraud-credit-exp,AutoML_fc4b5e57-5118-47f6-8be5-a071c1743d21,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
# submit script to AML
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
automl_best_run, automl_fitted_model = remote_run.get_output()

In [27]:
automl_best_run.properties['run_algorithm']

'VotingEnsemble'

In [10]:
automl_fitted_model.steps[1][1].estimators

[('3',
  Pipeline(memory=None,
           steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                  ('randomforestclassifier',
                   RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                          class_weight='balanced',
                                          criterion='gini', max_depth=None,
                                          max_features='log2',
                                          max_leaf_nodes=None, max_samples=None,
                                          min_impurity_decrease=0.0,
                                          min_impurity_split=None,
                                          min_samples_leaf=0.01,
                                          min_samples_split=0.01,
                                          min_weight_fraction_leaf=0.0,
                                          n_estimators=25, n_jobs=1,
                                          oob_score=True, random_state=No

In [12]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(automl_fitted_model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['3', '23', '28', '16', '7', '26'],
 'weights': [0.26666666666666666,
             0.06666666666666667,
             0.13333333333333333,
             0.06666666666666667,
             0.4,
             0.06666666666666667]}

3 - minmaxscaler
{'copy': True, 'feature_range': (0, 1)}

3 - randomforestclassifier
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 0.01,
 'min_samples_split': 0.01,
 'min_weight_fract

In [13]:
print(automl_fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                fit_intercept=True,
                                                                                                intercept_scaling=1,
                                                                                                l1_ratio=None,
                                                

In [14]:
display(automl_best_run.properties)

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-fraud-credit-exp\',\'compute_target\':\'auto-ml\',\'subscription_id\':\'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_fc4b5e57-5118-47f6-8be5-a071c1743d21_38","experiment_name":"capstone-fraud-credit-exp","workspace_name":"quick-starts-ws-142803","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group_name":"aml-quickstarts-142803"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '38',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_a

In [15]:
#TODO: Save the best model
import joblib

joblib.dump(automl_fitted_model,"best_automl_model.joblib")
automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env_v_1_0_0.yml')
automl_best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

All the information about the deployment of the model are coming from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python


In [16]:
automl_best_run.properties['model_name']

'AutoMLfc4b5e57538'

In [17]:
model = remote_run.register_model(model_name=automl_best_run.properties['model_name'])
# automl_best_run.register_model(model_name=automl_best_run.properties['model_name'], model_path='/model.pkl')
print(model.name, model.id, model.version, sep='\t')


AutoMLfc4b5e57538	AutoMLfc4b5e57538:1	1


In [18]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


env = automl_best_run.get_environment()

inference_config = InferenceConfig(entry_script='score.py',environment=env)

In [19]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model


deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
service = Model.deploy(ws, 'capstone-fraud-service', [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-15 22:00:42+00:00 Creating Container Registry if not exists..
2021-04-15 22:01:03+00:00 Registering the environment..
2021-04-15 22:01:05+00:00 Use the existing image.
2021-04-15 22:01:05+00:00 Generating deployment configuration.
2021-04-15 22:01:08+00:00 Submitting deployment to compute..
2021-04-15 22:01:15+00:00 Checking the status of deployment capstone-fraud-service..
2021-04-15 22:01:56+00:00 Checking the status of inference endpoint capstone-fraud-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
service.scoring_uri

'http://19e17946-733e-428a-81fb-3f513491770d.southcentralus.azurecontainer.io/score'

In [35]:
%run endpoint.py

{"result": [0, 0]}


In [36]:
import json

data = {"data":
        [
          {
            "Time": 0.0, "V1": 0.0, "V2": 0.0, "V3": 0.0, "V4": 0.0, "V5": 0.0, "V6": 0.0, "V7": 0.0, "V8": 0.0, "V9": 0.0, "V10": 0.0, "V11": 0.0, "V12": 0.0, "V13": 0.0, "V14": 0.0, "V15": 0.0, "V16": 0.0, "V17": 0.0, "V18": 0.0, "V19": 0.0, "V20": 0.0, "V21": 0.0, "V22": 0.0, "V23": 0.0, "V24": 0.0, "V25": 0.0, "V26": 0.0, "V27": 0.0, "V28": 0.0, "Amount": 0.0
          },
            {
            "Time": 0.0, "V1": 0.0, "V2": 0.0, "V3": 0.0, "V4": 0.0, "V5": 0.0, "V6": 0.0, "V7": 0.0, "V8": 0.0, "V9": 0.0, "V10": 0.0, "V11": 0.0, "V12": 0.0, "V13": 0.0, "V14": 0.0, "V15": 0.0, "V16": 0.0, "V17": 0.0, "V18": 0.0, "V19": 0.0, "V20": 0.0, "V21": 0.0, "V22": 0.0, "V23": 0.0, "V24": 0.0, "V25": 0.0, "V26": 0.0, "V27": 0.0, "V28": 0.0, "Amount": 0.0
          }
        ]
    }
# Convert to JSON string
input_data = json.dumps(data)

In [37]:
input_data

'{"data": [{"Time": 0.0, "V1": 0.0, "V2": 0.0, "V3": 0.0, "V4": 0.0, "V5": 0.0, "V6": 0.0, "V7": 0.0, "V8": 0.0, "V9": 0.0, "V10": 0.0, "V11": 0.0, "V12": 0.0, "V13": 0.0, "V14": 0.0, "V15": 0.0, "V16": 0.0, "V17": 0.0, "V18": 0.0, "V19": 0.0, "V20": 0.0, "V21": 0.0, "V22": 0.0, "V23": 0.0, "V24": 0.0, "V25": 0.0, "V26": 0.0, "V27": 0.0, "V28": 0.0, "Amount": 0.0}, {"Time": 0.0, "V1": 0.0, "V2": 0.0, "V3": 0.0, "V4": 0.0, "V5": 0.0, "V6": 0.0, "V7": 0.0, "V8": 0.0, "V9": 0.0, "V10": 0.0, "V11": 0.0, "V12": 0.0, "V13": 0.0, "V14": 0.0, "V15": 0.0, "V16": 0.0, "V17": 0.0, "V18": 0.0, "V19": 0.0, "V20": 0.0, "V21": 0.0, "V22": 0.0, "V23": 0.0, "V24": 0.0, "V25": 0.0, "V26": 0.0, "V27": 0.0, "V28": 0.0, "Amount": 0.0}]}'

In [38]:
service.run(input_data)


'{"result": [0, 0]}'

TODO: In the cell below, print the logs of the web service and delete the service

In [39]:
print(service.get_logs())

2021-04-15T22:01:51,728353400+00:00 - gunicorn/run 
2021-04-15T22:01:51,731267400+00:00 - rsyslog/run 
2021-04-15T22:01:51,748515400+00:00 - iot-server/run 
2021-04-15T22:01:51,778502900+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [40]:
service.delete()

In [41]:
service.state

'Deleting'